In [180]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import csv
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import matplotlib.pyplot as plt

### Get Vaccine Data from CDC

In [181]:
# Get Vaccinations Table from CDC website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://covid.cdc.gov/covid-data-tracker/#vaccinations")
vaccine_html = browser.html
browser.click_link_by_id("vaccinations-table-toggle")
cdc_html = browser.html
cdc_parsed = bs(cdc_html, "html.parser")
table = cdc_parsed.select("table")
browser.quit()

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\coled\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [182]:
# Read the CDC Vaccine HTML Table
vaccines_df = pd.read_html(str(table))[0]

In [183]:
# Change "New York State" to "New York" for data merging
vaccines_df.loc[vaccines_df["State/Territory/Federal Entity"] == "New York State", "State/Territory/Federal Entity"] = "New York"

In [184]:
# Rename "State/Territory/Federal Entity" column to "state" for data merging
vaccines_df = vaccines_df.rename(columns={"State/Territory/Federal Entity": "state"})

## Daily COVID-19 - NYT / Rearc

In [185]:
# Read Daily COVID-19 CSV from AWS S3 Bucket - Rearc / NY Times Data 
us_covid_cases_data = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-states/us-states.csv", stream=True)

In [186]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/us-covid-data.csv",  "wb") as file:
    file.write(us_covid_cases_data.content)

In [187]:
# Read US COVID Data CSV for data merging and cleaning 
us_covid_cases_df = pd.read_csv("../data/us-covid-data.csv")

# State Overview Data

In [188]:
# Get yesterday's date, which is the latest data 
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
latest_data = yesterday.strftime("%Y-%m-%d")

In [189]:
# Get latest total cases by state for latest date
us_total_cases_to_date_df = us_covid_cases_df.groupby(["date", "state"]).sum()
us_total_cases_to_date_df.reset_index(inplace=True)
us_total_cases_to_date_df = us_total_cases_to_date_df.loc[us_total_cases_to_date_df["date"] == latest_data]

In [190]:
# Read State Lat-Lon CSV and rename columns for easier cleaning and merging
state_latlons_df = pd.read_csv("../data/statelatlong.csv")
state_latlons_df.rename(columns={"State": "abbr", "Latitude": "lat", "Longitude": "lon", "City": "state"}, inplace=True)

In [191]:
# Read State Population CSV for cleaning and merging
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")

In [192]:
# Merge, rename, and focus on defined columns
state_overview_master_df = pd.merge(state_pop_df, state_latlons_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["state", "abbr", "lat", "lon", "population"]]

In [193]:
# Merge with us_total_cases_to_date_df and focus on defined columns
state_overview_master_df = pd.merge(state_overview_master_df, us_total_cases_to_date_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths"]]

In [194]:
# Merge with Vaccination data, rename columns, focus data columns
state_overview_master_df = pd.merge(state_overview_master_df, vaccines_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df.rename(columns={"Total Doses Administered by State where Administered": "total_doses_administered", "Doses Administered per 100k by State where Administered": "doses_administered_per_100k"})
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths", "total_doses_administered", "doses_administered_per_100k"]]

In [195]:
# Add Empty Column for New Calculation
state_overview_master_df["percent_vaccinated"] = ""

In [196]:
# Calcuation for percent vaccinated by state
for index, row in state_overview_master_df.iterrows():
    population = row["population"]
    vaccinated = row["total_doses_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_overview_master_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [197]:
# Convert percent vaccinated into a float
state_overview_master_df = state_overview_master_df.astype({"percent_vaccinated": 'float64'})

In [198]:
state_overview_master_df.to_csv("../data/state-master-data.csv")

In [199]:
state_overview_master_df["date"] = pd.to_datetime(state_overview_master_df["date"])

# County Overview Data

In [200]:
populations_county = pd.read_csv("../data/census-bureau-population-by-county.csv")
poverty_county = pd.read_csv("../data/poverty-and-median-household-income-data-by-us-county-2019.csv")
state_keys = pd.read_csv("../data/state-names-codes.csv")

In [201]:
poverty_county = poverty_county[["Postal Code", "county", "Poverty Estimate, All Ages", "Median Household Income"]]
poverty_county = poverty_county.rename(columns={"Postal Code": "code", "Poverty Estimate, All Ages": "poverty_est_all_ages", 
                                              "Median Household Income": "median_household_income"})
poverty_county["county"] = poverty_county["county"].str.rstrip()

In [202]:
state_keys = state_keys.rename(columns={"Code": "code", "State": "state"})
state_keys = state_keys[["state", "code"]]

In [203]:
poverty_county = pd.merge(poverty_county, state_keys, how="inner", on="code")
poverty_county.drop(columns={"state"}, inplace=True)

In [204]:
populations_county = populations_county.merge(state_keys, how="inner", on="state")
populations_county = populations_county[["code", "county", "state", "population_est"]]
populations_county["code"] = populations_county["code"].str.rstrip()
populations_county["county"] = populations_county["county"].str.rstrip()

In [205]:
merged_data = populations_county.merge(poverty_county, how='inner', left_on=["code", "county"], right_on=["code", "county"])

In [206]:
merged_data.dropna(how="any", inplace=True)

In [207]:
merged_data

,code,county,state,population_est,poverty_est_all_ages,median_household_income
0,AL,Alabama,Alabama,4903185,747478,51771
1,AL,Autauga,Alabama,55869,6723,58233
2,AL,Baldwin,Alabama,223234,22360,59871
3,AL,Barbour,Alabama,24686,5909,35972
4,AL,Bibb,Alabama,22394,4101,47918
...,...,...,...,...,...,...
3192,WY,Sweetwater,Wyoming,42343,3453,80639
3193,WY,Teton,Wyoming,23464,1396,98837
3194,WY,Uinta,Wyoming,20226,1699,70756
3195,WY,Washakie,Wyoming,7805,845,55122


### Retrieve County Cases Data

In [208]:
county_cases = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv", stream=True)

In [209]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/county-covid-data.csv",  "wb") as file:
    file.write(county_cases.content)

In [210]:
# Read US COVID Data CSV for data merging and cleaning 
us_county_cases = pd.read_csv("../data/county-covid-data.csv")

In [211]:
us_county_cases = us_county_cases.groupby(["date", "state", "county"]).sum()
us_county_cases.reset_index(inplace=True)

In [212]:
us_county_cases = us_county_cases.merge(merged_data, how="left")

In [213]:
us_county_cases.to_csv("../data/county-cases-daily-master.csv")

In [214]:
us_county_cases["date"] = pd.to_datetime(us_county_cases["date"])

In [215]:
latest_covid_cases = us_county_cases.loc[us_county_cases["date"] == latest_data]

In [216]:
latest_covid_cases.dropna(how="any", inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [217]:
for index, row in latest_covid_cases.iterrows():
    latest_covid_cases.at[index, "poverty_est_all_ages"] = row["poverty_est_all_ages"].replace('.', "0")
    latest_covid_cases.at[index, "median_household_income"] = row["median_household_income"].replace('.', "0")

In [218]:
latest_covid_cases.fillna(0, inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [219]:
latest_covid_cases["poverty_est_all_ages"] = latest_covid_cases["poverty_est_all_ages"].astype('int64')
latest_covid_cases["median_household_income"] = latest_covid_cases["median_household_income"].astype('int64')

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [220]:
education_data = pd.read_csv("../data/county-level-education-stats-2015-2019.csv")

In [221]:
education_data.rename(columns={"state": "code"}, inplace=True)

In [222]:
education_data["county"] = education_data["county"].str.rstrip()

In [223]:
latest_covid_cases = latest_covid_cases.merge(education_data, how='outer', left_on=["code", "county"], right_on=["code", "county"])

In [224]:
latest_covid_cases.dropna(inplace=True)

In [225]:
latest_covid_cases.to_csv("../data/county-cases-latest-master.csv")

## Daily Cases

In [226]:
# Convert Dates to pd.Datetimes
us_covid_cases_df["date"] = pd.to_datetime(us_covid_cases_df["date"])

In [227]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20524,2021-03-10,Virginia,51,589375,9849
20525,2021-03-10,Washington,53,349881,5159
20526,2021-03-10,West Virginia,54,134158,2330
20527,2021-03-10,Wisconsin,55,623150,7151


In [228]:
# Create Daily New Cases Dataframe
daily_new_cases_df = us_covid_cases_df.groupby("date").sum()

In [229]:
daily_new_cases_df.reset_index(inplace=True)

In [230]:
# Add new Empty Columns for Daily New Cases and Deaths
daily_new_cases_df["daily_new_cases"] = ""
daily_new_cases_df["daily_new_deaths"] = ""

In [231]:
previous_day_cases = 0
previous_day_deaths = 0
    
for index, row in daily_new_cases_df.iterrows():
    
    # Cases
    new_cases = row["cases"] - previous_day_cases
    previous_day_cases = row["cases"]
    daily_new_cases_df.at[index, "daily_new_cases"] = new_cases
    
    # Deaths
    new_deaths = row["deaths"] - previous_day_deaths
    previous_day_deaths = row["deaths"]
    daily_new_cases_df.at[index, "daily_new_deaths"] = new_deaths

In [236]:
daily_new_cases_df["3_month_immunity_est"] = daily_new_cases_df.rolling(100)["daily_new_cases"].sum()

In [237]:
daily_new_cases_df

,date,fips,cases,deaths,daily_new_cases,daily_new_deaths,3_month_immunity_est
0,2020-01-21,53,1,0,1,0,NaN
1,2020-01-22,53,1,0,0,0,NaN
2,2020-01-23,53,1,0,0,0,NaN
3,2020-01-24,70,2,0,1,0,NaN
4,2020-01-25,76,3,0,1,0,NaN
...,...,...,...,...,...,...,...
410,2021-03-06,1762,28993824,523970,56606,1459,16039328.0
411,2021-03-07,1762,29034160,524652,40336,682,15873778.0
412,2021-03-08,1762,29132673,525467,98513,815,15821145.0
413,2021-03-09,1762,29188505,527352,55832,1885,15740605.0


In [159]:
daily_new_cases_df["6_month_immunity_est"] = daily_new_cases_df.rolling(190)["daily_new_cases"].sum()

In [160]:
daily_new_cases_df = daily_new_cases_df.fillna(0)

In [161]:
daily_new_cases_df["date"] = daily_new_cases_df["date"].dt.strftime("%Y-%d-%m")

In [162]:
# daily_new_cases_df["date"]= pd.to_datetime(daily_new_cases_df["date"])
for index, row in daily_new_cases_df.iterrows():
    date = dt.datetime.strptime(row["date"], "%Y-%d-%m")
    date2 = dt.datetime.strftime(date, "%Y-%m-%d")
    daily_new_cases_df.at[index, "date"] = date2
#     print(date2)

In [163]:
daily_new_cases_df["date"] = pd.to_datetime(daily_new_cases_df["date"])

In [164]:
daily_new_cases_df.to_csv("../data/state-daily-stats.csv")

## Amazon RDS Updates

In [165]:
# Config Variables, and SQLalchemy
from config import endpoint, username, password
from sqlalchemy import create_engine

In [166]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [167]:
# 50 States & D.C. Data
state_overview_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)
engine.execute('ALTER TABLE master_table ADD PRIMARY KEY (state);')

In [168]:
# Daily Cases Data
daily_new_cases_df.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)
engine.execute('ALTER TABLE daily_new_cases ADD PRIMARY KEY (date);')

In [169]:
# County Data
latest_covid_cases.to_sql("county_cases_latest", index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE county_cases_latest ADD PRIMARY KEY (index);')

In [170]:
# Raw NTY Data
us_covid_cases_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE nyt_table ADD PRIMARY KEY (index);')